In [58]:
import pandas as pd

assignment_list = pd.read_csv('../../../nationalrat.csv', sep=";", usecols=['Name', ' Party(_ties)'])
assignment_list.columns = ['name', 'party']

In [59]:
all_parties = [item.strip() 
               for sublist in [party.strip().split(',') 
                               for party in assignment_list['party'].unique()] for item in sublist]
unique_parties = list(set(all_parties))
unique_parties.sort()

In [60]:
map_parties = {
    'andere': 'andere',
    'F': 'FPOe',
    'F-BZÖ': 'BZOe',
    'FPÖ': 'FPOe',
    'GRÜNE': 'GRUeNE',
    'ÖVP': 'OeVP',
    'SPÖ': 'SPOe',
    'KPÖ': 'KPOe',
    'BZÖ': 'BZOe',
    'NEOS': 'LIF-NEOS',
    'NEOS-LIF': 'LIF-NEOS',
    'PILZ': 'PILZ-JETZT',
    'STRONACH': 'STRONACH'
}

# MongoDB version

In [83]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017")

In [84]:
mydb = myclient["dylen"]
mycol = mydb["generalNet"]

In [91]:
# reset
for party in unique_parties:
    if party in map_parties.keys():
        myquery_party = {"entity": map_parties[party], 
                         "available_years": {"$exists": True}}

        mycol.update_many(myquery_party, {'$set': {'speakers': []}})

In [92]:
speakers_per_party = dict()

for idx, row in assignment_list.iterrows():
    myquery = { "entity_name": f"{row['name']}", "available_years": {"$exists": True}}
    
    for result in mycol.find(myquery):
        speaker_return = result['entity_name']
        parties = [party.strip() for party in row['party'].split(',')]
        
        for party in parties:
            if party in map_parties.keys():
                party = map_parties[party]
            else:
                party = 'andere'
            
            if party in speakers_per_party.keys():
                speakers_per_party[party].append(speaker_return)
            else:
                speakers_per_party[party] = [speaker_return]


In [ ]:
for assignment in speakers_per_party:
    myquery_party = {"entity": assignment, 
                     "entity_name": {"$exists": False}, 
                     "available_years": {"$exists": True}}
        
    list_politicians = list(set(speakers_per_party[assignment]))
    list_politicians.sort()
    
    for politician in list_politicians:
        mycol.update_one(myquery_party, {'$push': {'speakers': politician}})

# JSON version

In [61]:
import glob as glob

data_walk = glob.glob('../../../ParlAT_general/party/**/time_series.json')

In [62]:
import json as json_obj

for path in data_walk:
    print(path)
    f = open(path, 'r')
    content = f.read()
    json_content = json_obj.loads(content)
        
    json_content['speakers'] = []
    
    fw = open(path, 'w')
    json_obj.dump(json_content, fw)

../../../ParlAT_general/party/STRONACH/time_series.json
../../../ParlAT_general/party/GRUeNE/time_series.json
../../../ParlAT_general/party/andere/time_series.json
../../../ParlAT_general/party/LIF-NEOS/time_series.json
../../../ParlAT_general/party/SPOe/time_series.json
../../../ParlAT_general/party/FPOe/time_series.json
../../../ParlAT_general/party/parteilos/time_series.json
../../../ParlAT_general/party/OeVP/time_series.json
../../../ParlAT_general/party/BZOe/time_series.json


In [63]:
speakers_per_party = dict()

for idx, row in assignment_list.iterrows():
    speaker_return = row['name']
    parties = [party.strip() for party in row['party'].split(',')]
        
    for party in parties:
        if party in map_parties.keys():
            party = map_parties[party]
        else:
            party = 'andere'
            
        if party in speakers_per_party.keys():
            speakers_per_party[party].append(speaker_return)
        else:
            speakers_per_party[party] = [speaker_return]

In [64]:
speakers_per_party.keys()

dict_keys(['SPOe', 'andere', 'FPOe', 'BZOe', 'OeVP', 'LIF-NEOS', 'GRUeNE', 'PILZ-JETZT', 'STRONACH', 'KPOe'])

In [65]:
for assignment in speakers_per_party:
    list_politicians = list(set(speakers_per_party[assignment]))
    list_politicians.sort()
    
    path = f'../../../ParlAT_general/party/{assignment}/time_series.json'
    try:
        f = open(path, 'r')
        content = f.read()
        json_content = json_obj.loads(content)

        json_content['speakers'] = list_politicians
        with open(path, 'w') as fw:
            json_obj.dump(json_content, fw)
    except:
        print('No file found for', assignment)

No file found for PILZ-JETZT
{'jaccard_similarity': {'previous_year': [0.420389461626575, 0.37677419354838715, 0.42542787286063566, 0.5335868187579214, 0.5607064017660044], 'first_year': [0.420389461626575, 0.39276807980049877, 0.45504385964912286, 0.5668316831683169, 0.4980694980694981], 'last_year': [0.4980694980694981, 0.4580384226491405, 0.5052083333333333, 0.4418837675350702, 0.5607064017660044]}, 'rankdcg_similarity': {'previous_year': [0.5942453512874363, 0.4986420220305284, 0.47195239951898715, 0.44944948587443195, 0.548201697345653], 'first_year': [0.5942453512874363, 0.6493485607967726, 0.45790745077355705, 0.46273571045944084, 0.6736044405430668], 'last_year': [0.6736044405430668, 0.5501502287116785, 0.595554492537084, 0.4046344438155195, 0.548201697345653]}, 'frobenius_similarity': {'previous_year': [77.73601453471251, 70.78305600858414, 73.13294374511999, 68.11380312755708, 79.04599287453658], 'first_year': [77.73601453471251, 72.66927056421726, 79.784686884624, 68.4633915

<-- Set mongod export PATH
`ls -1 */**/*.json | while read jsonfile; do mongoimport --uri "mongodb://localhost:27017/dylen" -c generalNet --file $jsonfile --type json;done`

# Cleanup

In [80]:
speakers_per_party_cleaned = dict()

for assignment in speakers_per_party:
    print(assignment)
    myqueryspeakers = {'entity': assignment, 'available_years': {'$exists': True}}
    
    for result in mycol.find(myqueryspeakers):
        for speaker in result['speakers']:
            speakerQry = {'entity_name': speaker, 'available_years': {'$exists': True}}
            
            print(speaker)
            count = 0
            for speak_result in mycol.find(speakerQry):
                count = count + 1
            
            if count == 0:
                print('...Not found!')
            else:
                if assignment in speakers_per_party_cleaned.keys():
                    speakers_per_party_cleaned[assignment].append(speaker)
                else:
                    speakers_per_party_cleaned[assignment] = [speaker]
                print('... Found!')

SPOe
Ablinger Sonja
...Not found!
Abraham Gerhard
...Not found!
Achs Matthias
...Not found!
Afritsch Josef
...Not found!
Aigner Edmund
...Not found!
Alberer Alois
...Not found!
Albrecht Anneliese
...Not found!
Androsch Hannes, Dipl.-Kfm. Dr.
...Not found!
Androsch Maurice, Ing.
...Not found!
Antoni Dieter, Dr.
... Found!
Antoni Konrad
...Not found!
Appel Rudolf
...Not found!
Astl Johann
...Not found!
Auer Josef, Mag.
...Not found!
Babanitz Franz
...Not found!
Bacher Walter
...Not found!
Bauer Hannes, Dipl.-Kfm. Dr.
...Not found!
Bauer Johann, Dipl.-Kfm. Dr. siehe Bauer Hannes, Dipl.-Kfm. Dr.
... Found!
Bauer Sophie
...Not found!
Bayr Petra, MA MLS
... Found!
Becher Ruth, Mag.
...Not found!
Bechinie Robert, Dipl.-Kfm. Dr.
...Not found!
Benya Anton
...Not found!
Binder Gabriele siehe Binder-Maier Gabriele
...Not found!
Binder-Maier Gabriele
...Not found!
Bisjak Paul
...Not found!
Blecha Karl
...Not found!
Blümel Johann
...Not found!
Brachmann Hans
...Not found!
Braun Helmut
...Not found!

...Not found!
Kräuter Günther, Dr.
...Not found!
Kräutl Adolf
...Not found!
Kuba Heinrich
...Not found!
Kubitschek Maria, Mag.
...Not found!
Kucharowits Katharina
...Not found!
Kucher Philip
... Found!
Kummerer Werner, Dipl.-Ing.
...Not found!
Kunst Josef, Ing.
...Not found!
Kunstätter Harald
...Not found!
Kuntzl Andrea, Mag.
... Found!
Kuzdas Hubert, Ing. Mag.
...Not found!
Kysela Karl
...Not found!
Köck Franz
...Not found!
Köfer Gerhard
...Not found!
Königsberger-Ludwig Ulrike
...Not found!
Körner Theodor, Dr. h.c.
...Not found!
Köteles Johann
...Not found!
Kövari Susanne
...Not found!
Lacina Ferdinand, Dipl.-Kfm.
...Not found!
Lackner Hermann
...Not found!
Lackner Manfred
...Not found!
Lagger Hans
...Not found!
Laimer Robert
...Not found!
Lanc Erwin
...Not found!
Lapp Christine, Mag., MA
...Not found!
Lausecker Karl
...Not found!
Lehr Friedrich
...Not found!
Leichtfried Jörg, Mag.
...Not found!
Leikam Anton
... Found!
Leithenmayr Hermann
...Not found!
Lenzi Josef, Dr.
...Not found!


...Not found!
Hanusch Ferdinand
...Not found!
Hareter Alexander
...Not found!
Hartleb Karl
...Not found!
Hartmann Josef
...Not found!
Haselsteiner Hans Peter, Dr.
... Found!
Hasenauer Bartholomäus
...Not found!
Haueis Alois
...Not found!
Hauser Johann Nepomuk
...Not found!
Hautmann Marie
...Not found!
Heigl Josef
...Not found!
Heinl Eduard, Dr. h.c.
...Not found!
Heinz Karl
...Not found!
Heitzinger Johann
...Not found!
Hermann Hermann
...Not found!
Herzele Max
...Not found!
Heuberger August
...Not found!
Hirsch Ernst
...Not found!
Hofer Franz, Dr.
...Not found!
Hofer Hans
...Not found!
Hohenberg Karl
...Not found!
Hollersbacher Josef
...Not found!
Holzinger Daniela, BA siehe Holzinger-Vogtenhuber Daniela, BA
...Not found!
Holzinger-Vogtenhuber Daniela, BA
...Not found!
Honner Franz
...Not found!
Horvatek Norbert
...Not found!
Hosch Kaspar
...Not found!
Hryntschak Alexander, Dr.
...Not found!
Huber Gerhard
... Found!
Hubmann Georg
...Not found!
Hueber Anton
...Not found!
Hueber Franz, D

...Not found!
Widholz Laurenz
...Not found!
Wiedenhofer Josef
...Not found!
Wieninger Josef
...Not found!
Wiesmaier Josef
...Not found!
Wimmer Elias
...Not found!
Wimmer Johann
...Not found!
Wimmer Matthias
...Not found!
Winkler Franz, Ing.
...Not found!
Winsauer Ernst, Ing.
...Not found!
Winter Susanne, Dr.
...Not found!
Witternigg Josef
...Not found!
Witzany Franz
...Not found!
Witzany Hans
...Not found!
Wollek Richard
...Not found!
Wollinger Karl
...Not found!
Wotawa August, Dr.
...Not found!
Wunsch Oskar
...Not found!
Zangel Josef
...Not found!
Zarboch Rudolf
...Not found!
Zauner Josef
...Not found!
Zehetgruber Franz
...Not found!
Zeidler Viktor, Dr.
...Not found!
Zeillinger Gustav
...Not found!
Zeininger Sylvester
...Not found!
Zelenka Franz
...Not found!
Zinggl Wolfgang, Mag. Dr.
... Found!
Zingl Josef
...Not found!
Zwanzger Johann
...Not found!
Zwenk Heinrich
...Not found!
FPOe
Achatz Anna Elisabeth
...Not found!
Achleitner Elke, Dipl.-Ing.
... Found!
Amesbauer Hannes, BA
...Not

... Found!
Widmann Rainer, Mag.
... Found!
Windholz Ernest
... Found!
Wittauer Klaus
... Found!
OeVP
Aichhorn Karl
...Not found!
Altenburger Erwin
...Not found!
Amon Werner siehe Amon Werner, MBA
...Not found!
Amon Werner, MBA
... Found!
Amtmann Heinrich, Ing.
...Not found!
Arthold Josef
...Not found!
Aubauer Gertrude, Mag.
...Not found!
Auer Jakob
... Found!
Auer Klaus Hubert, Dipl.-Ing.
...Not found!
Babitsch Leopold, Dipl.-Ing.
...Not found!
Bandion Karl
...Not found!
Bartenstein Martin, Dr.
... Found!
Bassetti Luis, Dipl.-Kfm. Dr.
...Not found!
Bauer Franz
...Not found!
Bauer Franz, Dr.
...Not found!
Bauer Rosemarie
... Found!
Baumgartner Angela
...Not found!
Baumgartner-Gabitzer Ulrike, Dr.
...Not found!
Bayer Johanna, Dipl.-Ing. Dr.
...Not found!
Bayr Anton
...Not found!
Bergmann Kurt
...Not found!
Bergsmann Felix
...Not found!
Berl Franz, Dipl.-Ing.
...Not found!
Berlakovich Nikolaus, Dipl.-Ing.
... Found!
Blenk Wolfgang, Dr.
...Not found!
Bleyer Wilhelm
...Not found!
Bock Fritz

...Not found!
Langreiter Hans, Mag.
...Not found!
Lanner Sixtus, Dr.
...Not found!
Lechner Franz, Dipl.-Ing. Dr.
...Not found!
Ledolter Johann
...Not found!
Leibenfrost Albert, Dr.
...Not found!
Leimböck Ernst
...Not found!
Leiner Günther, Dr.
...Not found!
Leisser Franz
...Not found!
Leitner Alois, Dipl.-Ing. Dr.
...Not found!
Leitner Peter
...Not found!
Lentsch Edeltraud
...Not found!
Letmaier Josef, Ing.
...Not found!
Lettenbichler Josef, Mag.
...Not found!
Lexer Reinhold
...Not found!
Lichal Robert, Dr.
...Not found!
Liechtenstein Vincenz, Dr.
...Not found!
Lindinger Klaus, Ing., BSc
...Not found!
Lins Herbert
...Not found!
Linsbauer Franz
...Not found!
Loos Johann
...Not found!
Lopatka Reinhold, Dr.
... Found!
Ludwig Eduard, Dr.
...Not found!
Luhamer Josef
...Not found!
Lukesch Dieter, Dipl.-Vw. Dr.
... Found!
Lussmann Hermann
...Not found!
Löffler Robert, Dipl.-Kfm.
...Not found!
Machne Helga
...Not found!
Machunze Erwin
...Not found!
Mackowitz Rudolf
...Not found!
Maderthaner Le

...Not found!
Windisch Franz, Ing.
...Not found!
Winkler Josef, Ing.
...Not found!
Winsauer Werner, Dipl.-Ing.
...Not found!
Winzig Angelika, Dr.
...Not found!
Withalm Hermann, Dr.
...Not found!
Witrisal Alois
...Not found!
Wolf Johann
...Not found!
Wolfmayr Andrea, Dr. siehe Wolfmayr Andrea, Mag. Dr.
...Not found!
Wolfmayr Andrea, Mag. Dr.
...Not found!
Wunder Gottfried
...Not found!
Wurmitzer Georg
... Found!
Wöginger August
...Not found!
Wölfler Alois, Mag.
...Not found!
Wührer Alois
...Not found!
Zakostelsky Andreas, Mag.
... Found!
Zarits Christoph
...Not found!
Zernatto Christof, Dr.
...Not found!
Zittmayr Hermann, Dipl.-Ing. Dr.
...Not found!
Zweytick Johannes
...Not found!
LIF-NEOS
Alm Nikolaus, Mag.
... Found!
Bernhard Michael
...Not found!
Doppelbauer Karin, Dipl.-Ing.
...Not found!
Gamon Claudia, MSc (WU)
... Found!
Griss Irmgard, Dr.
...Not found!
Hable Rainer, Dr.
... Found!
Hoyos-Trauttmansdorff Douglas
...Not found!
Krisper Stephanie, Dr.
...Not found!
Loacker Gerald, Ma